# Footwear Inventory Example
Created 4/27/2022 by

Timothy Del Green<br />
1-256-335-0378<br />
tdgreen@outlook.com<br />
https://www.linkedin.com/in/timothy-del-green

## Notebook Setup

### Imports

In [119]:
import warnings
warnings.filterwarnings('ignore')

## SQL query for boot information

In [122]:
%%capture --no-display

%%sql sql_query <<

SELECT
    B.ICITEM,
    B.ICDSC1,
    B.ICDSC2

FROM R50FILES.VINITMB

LEFT JOIN R50FILES.VINITEM AS B
    ON IFCOMP = 1
    AND IFITEM = B.ICITEM

WHERE IFCOMP = 1
    AND IFQOH != 0
    AND IFDIV = 2 
    AND IFCLS = 1 
    AND IFSCLS = 10 
    AND IFSSCLS = 1

GROUP BY B.ICITEM, B.ICDSC1, B.ICDSC2

### Results of SQL query

In [123]:
sql_query

,icitem,icdsc1,icdsc2
0,BATA86020XXL,"BOOT,10""OVER SHOE XXL 86020XXL",
1,NSPCHM-000A-10,"MUCK CHORE, MID WORK BOOT",
2,NSPCHM-000A-11,"MUCK CHORE, MID WORK BOOT",
3,NSPCHM-000A-12,"MUCK CHORE, MID WORK BOOT",
4,NSPCHM-000A-13,"MUCK CHORE, MID WORK BOOT",
...,...,...,...
670,ONG84086-14,"BOOTS, STEEL TOE DURAPRO SZ 14",XCP CHEVRON SOLE
671,ONG84086-15,"BOOT,DURAPRO XCP, STEEL TOE",
672,WARSRB7605-11.0W,"SAFETY BOOTS, REEBOK, EH RATED","INTERNAL MET, SIZE 11 WIDE"
673,FOOT7591-10.5M,"6"" SAFETY BOOT, HAMMER BROWN","EH, WATERPROOF, SIZE 10.5M"


## List of manufacturer item numbers to include in final output

In [124]:
item_list = [
    '8225',
    '8125',
    'RK664',
    'IA5061',
    'RB7755',
    'RB4041',
    'RB4142',
    'RB142',
    'RB431',
    'FS2650',
    'RK751',
    '2002XP',
]

## Shoe Model

In [125]:
def regex_sizes(df_, col, extract_col):
    """Raw string with extracted item sizes

    Args:
        df_ (DataFrame): The current state of the piped DataFrame    
        col (str): The column name to create
        extract_col (str): The column containing the item numbers

    Returns:
        DataFrame: DataFrame with appended columns
    """
    regex_list = [
        # Item number like : BATA86020XXL
        r'(?:L[/]XL|XXL|XL|L)$',

        # Item number like : LPR2002XP-10.5EEE
        r'(?<=[-])(?:[0-9.]{1,4}\w*)$'
    ]

    # Searches for patterns in regex_list and creates column with matches
    df_[col] = df_[extract_col].str.extract('(' + '|'.join(regex_list) + ')')

    return df_

In [126]:
def size_cleanup(df_, col, default_col):
    """Cleans up the item sizes extracted with regex

    Args:
        df_ (DataFrame): Current state of the piped DataFrame
        col (str): Column name to create
        default_col (str): Column with regex extracted item sizes

    Returns:
        DataFrame: DataFrame with appended columns
    """
    conditions = [
        # Trailing zero after size
        df_[default_col].str.contains('(1\d{1}0)') == True,
        # Leading zero before size
        df_[default_col].str.contains('(0\d{1,2})') == True,
        # Decimal-zero at end of numeriical size
        df_[default_col].str.contains('(\d{1,2}[.]0)') == True
    ]

    choices = [
        # Remove trailing zero after size, like '120' --> '12'
        df_[default_col].str.slice(start=0, stop=2),
        # Remove leading zero from size, like '09.5' --> '9.5'
        df_[default_col].str.strip('0'),
        # Remove decimal-zero at end of numeriical size, like '10.0' --> '10'
        df_[default_col].str.replace('.0', '', regex=False)
    ]

    df_[col] = np.select(conditions, choices, default=df_[default_col])

    return df_

In [127]:
def boot_gender(df_, col, col1, col2):
    """Determines the sex of each item

    Args:
        df_ (DataFrame): The current state of the piped DataFrame
        col (str): The column name to create
        col1 (str): Item Description 1
        col2 (str): Item Description 2

    Returns:
        DataFrame: DataFrame with appended column
    """
    df_[col] = np.where(
        # If either description field contains 'WOMEN', then 'Women'
        (df_[col1].str.contains('WOMEN')) |
        (df_[col2].str.contains('WOMEN')),
        'Women',
        'Men'
    )

    return df_

In [128]:
def numerical_boot_sizes(df_, col, size_col):
    """Extracts the numberical boot size from extracted info

    Args:
        df_ (DataFrame): Current state of piped DataFrame
        col (str): Name of column to create
        size_col (str): Column name with extracted sizes

    Returns:
        DataFrame: DataFrame with appended column
    """
    df_[col] = df_[size_col].str.extract(r'([0-9.]*)(?=\D*$)')

    return df_

In [129]:
def char_boot_sizes(df_, col, size_col):
    """Extracts non-numerical boot sizes

    Args:
        df_ (DataFrame): Current State of piped DataFrame
        col (str): Name of column to create
        size_col (str): Column name with extractes sizes

    Returns:
        DataFrame: DataFrame with appended column
    """
    df_[col] = df_[size_col].str.extract(r'(L[/]XL|XXL|2XL|XL|L)$')

    return df_

In [130]:
def gender_sizes(df_, col, search_gender, search_col, size_col):
    """Determines the item size based on gender

    Args:
        df_ (DataFrame): Current state of piped DataFrame
        col (str): name of column to create
        search_gender (str): Gender to search for
        search_col (str): column name with item genders
        size_col (str): Column name with extracted sizes

    Returns:
        DataFrame: DataFrame with appended column
    """
    df_[col] = np.where(
        df_[search_col] == search_gender,
        df_[size_col],
        None
    )

    return df_

### Shoe Width Reference

<table id="tablepress-363" class="tablepress tablepress-id-363 tablepressThree">
    <thead>
        <tr class="row-1 odd">
            <th class="column-1">Letter</th>
            <th class="column-2">Width</th>
            <th class="column-3">Abbreviation</th>
        </tr>
    </thead>
    <tbody class="row-hover">
        <tr class="row-2 even">
            <td class="column-1">B</td>
            <td class="column-2">Narrow</td>
            <td class="column-3">N</td>
        </tr>
        <tr class="row-3 odd">
            <td class="column-1">D</td>
            <td class="column-2">Medium or Average</td>
            <td class="column-3">M or A</td>
        </tr>
        <tr class="row-4 even">
            <td class="column-1">2E or EE</td>
            <td class="column-2">Wide</td>
            <td class="column-3">W</td>
        </tr>
        <tr class="row-5 odd">
            <td class="column-1">4E or EEEE</td>
            <td class="column-2">Extra-Wide</td>
            <td class="column-3">WW or XW or EW</td>
        </tr>
    </tbody>
</table>

- https://www.blitzresults.com/en/shoe-size-width/



In [131]:
def boot_width(df_, col, size_col):
    """Creates a column containing boot width letter

    Args:
        df_ (DataFrame): Current state of piped DataFrame
        col (str): Name of column to create
        size_col (str): Column name with extracted item sizes

    Returns:
        DataFrame: DataFrame with appended column
    """
    df_[col] = (
        df_[size_col].str.extract('(\D*$)')
            # Width for non-numerical boot sizes is None
            .replace(to_replace=r'(L[/]XL|2XL|XXL|XL)', value=None, regex=True)
            .replace(to_replace='M', value='D')
            .replace(to_replace='W', value='EE')
            .replace(to_replace='EW', value='EEEE')
    )
    
    return df_

In [132]:
def shoe_model(sql_query):
    """Extracts boot information from SQL query. Corresponds with 'Shoe Model' sheet from template.

    Args:
        sql_query (DataFrame): DataFrame containing query data from database

    Returns:
        DataFrame: DataFrame with boot information
    """
    return (
        sql_query[sql_query['icitem'].str.contains('|'.join(item_list), case=False, regex=True)]
            .pipe(regex_sizes, 'item_sizes', 'icitem')
            .pipe(size_cleanup, 'item_sizes_clean', 'item_sizes')
            .pipe(numerical_boot_sizes, 'numerical_size', 'item_sizes_clean')
            .pipe(char_boot_sizes, 'char_size', 'item_sizes_clean')
            .pipe(boot_gender, 'Gender', 'icdsc1', 'icdsc2')
            .pipe(gender_sizes, 'MaleSize', 'Men', 'Gender', 'numerical_size')
            .pipe(gender_sizes, 'FemaleSize', 'Women', 'Gender', 'numerical_size')
            .pipe(boot_width, 'Width', 'item_sizes_clean')
            .assign(
                icitem = lambda df_ : df_['icitem'].str.split('-').str[0],
                TempShoeID = '',
                Scale = 'US',
                Id = '',
                SKU = '',
                UPC = '',
                isAvailable = '',
                isDiscontinued = '',
            )
            .drop(columns={
                'icdsc1',
                'icdsc2',
                'item_sizes',
                'item_sizes_clean',
                'numerical_size',
                'char_size',
            })
            .rename(columns={
                'TempShoeID': 'Id (Temp Shoe ID)',
                'icitem'    : 'Style'
            })
            .reindex(columns=[
                'Id',
                'SKU',
                'UPC',
                'Id (Temp Shoe ID)',
                'Style',
                'Gender',
                'MaleSize',
                'FemaleSize',
                'Width',
                'Scale',
                'isAvailable',
                'isDiscontinued',
            ])
            .sort_values(by='Style')
            .reset_index(drop=True)
    )


### Shoe Model Output

In [133]:
shoe_model(sql_query)

,Id,SKU,UPC,Id (Temp Shoe ID),Style,Gender,MaleSize,FemaleSize,Width,Scale,isAvailable,isDiscontinued
0,,,,,FOOT8125,Women,None,7.5,D,US,,
1,,,,,FOOT8125,Women,None,7,D,US,,
2,,,,,FOOT8125,Women,None,6.5,D,US,,
3,,,,,FOOT8125,Women,None,6,D,US,,
4,,,,,FOOT8225,Men,14,None,D,US,,
...,...,...,...,...,...,...,...,...,...,...,...,...
150,,,,,WARSRK751,Men,7.5,None,D,US,,
151,,,,,WARSRK751,Men,8,None,D,US,,
152,,,,,WARSRK751,Men,8.5,None,D,US,,
153,,,,,WARSRK751,Men,9,None,D,US,,


## Shoe Properties

Resources
- https://constructioninformer.com/alloy-toe-vs-steel-toe/
- https://honeywell-sps.force.com/s/article/What-Does-ASTM-F2413-18-Mean

In [134]:
def properties_select(df_):
    """Determines several boolean properties of boots

    Args:
        df_ (DataFrame): Current state of piped DataFrame

    Returns:
        DataFrame: DataFrame with appended columns
    """
    
    # Dictionary with column names and search text
    properties_dict = {
        'PenetrationResistant'  : ['PENETRATION RESISTANT'],
        'SteelToe'              : ['STEEL TOE'],
        'Metatarsal'            : ['METATARSAL'],
        'OilResistant'          : ['OIL RESISTANT'],
        'HeatResistant'         : ['HEAT RESISTANT'],
        'SlipResistant'         : ['SLIP RESISTANT', 'SR'],
        'ElectricalResistant'   : ['ELECTRICAL RESISTANT', 'EH'],
        'Waterproof'            : ['WATERPROOF', 'WP'],
        'LadderSupport'         : ['LADDER SUPPORT'],
        'Insulated'             : ['INSULATED'],
        'StaticDissipative'     : ['STATIC DISSIPATIVE', 'SD', 'SD35'],
        'Laced'                 : ['LACED'],
        'Slipon'                : ['SLIPON'],
        'AlloyToe'              : ['ALLOY TOE'],
        'CompositeToe'          : ['COMPOSITE TOE', 'COMP TOE'],
        'CarbonToe'             : ['CARBON TOE'],
        'ZipUp'                 : ['ZIP UP'],
        'nonMetal'              : ['NON METAL'],
        'MadeInUsa'             : ['MADE IN USA'],
        'BuiltInUsa'            : ['BUILT IN USA'],
        'DefinedHeel'           : ['DEFINED HEEL'],
        'ninetyDegreeHeel'      : ['NINETY DEGREE HEEL'],
        'aluminumToe'           : ['ALUMINUM TOE'],
        'conductive'            : ['CONDUCTIVE'],
        'nursing'               : ['NURSING'],
        'hospitality'           : ['HOSPITALITY'],
        'industrial'            : ['INDUSTRIAL'],
        'oshaApproved'          : ['OSHA APPROVED'],
        'abrasionResistant'     : ['ABRASION RESISTANT'],
        'chemicalResistant'     : ['CHEMICAL RESISTANT'],
        'workAndSafetyBoots'    : ['WORK AND SAFETYBOOTS'],
        'workAndSafetySneakers' : ['WORK AND SAFETY SNEAKERS'],
        'safetyToe'             : ['SAFETY TOE'],
        'openToeHeel'           : ['OPEN TOE HEEL'],
        'taaCompliant'          : ['TAA COMPLIANT'],
        'sixInchBoot'           : ['SIX INCH BOOT', "6''", '6"'],
        'fullLeatherUpper'      : ['FULL LEATHER UPPER'],
        'mixedLeatherUpper'     : ['MIXED LEATHER UPPER']
    }

    # For each property, act if either item description contains any of
    # that property's search terms 
    for key, val in properties_dict.items():
        df_[key] = np.where(
            (df_['icdsc1'].str.contains('|'.join(val), case=False, regex=True)) |
            (df_['icdsc2'].str.contains('|'.join(val), case=False, regex=True)),
            'Yes',
            'No' 
        )

    return df_


In [135]:
def shoe_properties(sql_query):
    """Extracts boot information from SQL query. Corresponds with 'Shoe Properties' sheet from template

    Args:
        sql_query (DataFrame): DataFrame containing SQL query results from database

    Returns:
        DataFrame: DataFrame with boot property data
    """
    return (
        sql_query[sql_query['icitem'].str.contains('|'.join(item_list), 
                                                    case=False, regex=True)]
            .assign(
                # item number striped of hypen and following characters
                icitem = lambda df_ : df_['icitem'].str.split('-').str[0],
                TempShoeID = '',
                Id = '',
                Material = '',
                Tread = '',
            )
            .pipe(properties_select)
            .drop(columns={
                'icdsc1',
                'icdsc2',
            })
            .drop_duplicates()
            .rename(columns={
                'TempShoeID': 'Id (Temp Shoe ID)',
                'icitem'    : 'Style',
            })
            .reindex(columns=[
                'Id', 
                'Id (Temp Shoe ID)', 
                'Style', 
                'Material', 
                'Tread',
                'PenetrationResistant', 
                'SteelToe', 
                'Metatarsal', 
                'OilResistant',
                'HeatResistant', 
                'SlipResistant', 
                'ElectricalResistant', 
                'Waterproof',
                'LadderSupport', 
                'Insulated', 
                'StaticDissipative', 
                'Laced', 
                'Slipon',
                'AlloyToe', 
                'CompositeToe', 
                'CarbonToe', 
                'ZipUp', 
                'nonMetal',
                'MadeInUsa', 
                'BuiltInUsa', 
                'DefinedHeel', 
                'ninetyDegreeHeel',
                'aluminumToe', 
                'conductive', 
                'nursing', 
                'hospitality', 
                'industrial',
                'oshaApproved', 
                'abrasionResistant', 
                'chemicalResistant',
                'workAndSafetyBoots', 
                'workAndSafetySneakers', 
                'safetyToe',
                'openToeHeel', 
                'taaCompliant', 
                'sixInchBoot',
                'fullLeatherUpper',
                'mixedLeatherUpper',
            ])
            .sort_values(by='Style')
            .reset_index(drop=True)
    )


### Shoe Properties Output

In [136]:
shoe_properties(sql_query)

,Id,Id (Temp Shoe ID),Style,Material,Tread,PenetrationResistant,SteelToe,Metatarsal,OilResistant,HeatResistant,...,abrasionResistant,chemicalResistant,workAndSafetyBoots,workAndSafetySneakers,safetyToe,openToeHeel,taaCompliant,sixInchBoot,fullLeatherUpper,mixedLeatherUpper
0,,,FOOT8125,,,No,Yes,No,No,No,...,No,No,No,No,No,No,No,Yes,No,No
1,,,FOOT8225,,,No,Yes,No,No,No,...,No,No,No,No,No,No,No,Yes,No,No
2,,,LPR2002XP,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
3,,,WARSFS2650,,,No,Yes,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4,,,WARSIA5061,,,No,Yes,No,No,No,...,No,No,No,No,No,No,No,Yes,No,No
5,,,WARSRB142,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
6,,,WARSRB4041,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
7,,,WARSRB4142,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
8,,,WARSRB431,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
9,,,WARSRB4310,,,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No


## Shoe Table

In [137]:
def color_select(df_):
    """Selects boot color from item description

    Args:
        df_ (DataFrame): Current state of DataFrame from pipe

    Returns:
        DataFrame: DataFrame with appended column
    """
    colors_dict = {
        'Brown'         : ['BRWN','BROWN','DRK BRWN'],
        'Black/Grey'    : ['BLK/GRY'],
        'Black'         : ['BLACK'],
    }
    
    conditions = []
    choices = []

    # For each property, act if the item description contains any of
    # that property's search terms
    for key, val in colors_dict.items():
        conditions.append(
            df_['icdsc2'].str.contains('|'.join(val), case=False, regex=True)
        )
        choices.append(key)
    
    df_['Color'] = np.select(conditions, choices, default=np.nan)

    return df_


In [138]:
def manufacturer_select(df_):
    """Selects boot manufacturer from item description

    Args:
        df_ (DataFrame): Current state of DataFrame from pipe

    Returns:
        DataFrame: DataFrame with column appended
    """
    manu_dict = {
        'Reebok'            : ['WARSRB'],
        'Avenger'           : ['FOOT'],
        'Royer'             : ['LPR'],
        'Rockport'          : ['WARSRK'],
        'Florsheim Work'    : ['WARSFS'],
        'Iron Age'          : ['WARSIA'],
    }
    
    conditions = []
    choices = []

    # For each property, act if the item description contains any of
    # that property's search terms
    for key, val in manu_dict.items():
        conditions.append(
            df_['icitem'].str.contains('|'.join(val), case=False, regex=True)
        )
        choices.append(key)
    
    df_['Manufacturer'] = np.select(conditions, choices, default=np.nan)

    df_['Brand'] = df_['Manufacturer']

    return df_

In [139]:
def image_merge(df_):
    """Merge column with image URLs for each item number

    Args:
        df_ (DataFrame): Current state of DataFrame from pipe

    Returns:
        DataFrame: DataFrame with merged columns
    """
    image_dict = {
        'LPR2002XP'     : 'https://cdn.shopify.com/s/files/1/0569/8197/9308/products/12002XP_BOTH_750x.jpg?v=1640296938',
        'FOOT8225'      : 'https://cdn.shopify.com/s/files/1/1212/1594/products/A7225-2017-2-stack_1024x1024@2x.jpg?v=1593207589',
        'FOOT8125'      : 'https://cdn.shopify.com/s/files/1/1212/1594/products/A7125-stack_1024x1024@2x.jpg?v=1593207847',
        'WARSRK664'     : 'https://cdn11.bigcommerce.com/s-deojk6ur3d/images/stencil/1280x1280/products/655/3063/gr1wqumwd9lyhkz03i44__46782.1651182542.png?c=1',
        'WARSRK6640'    : 'https://cdn11.bigcommerce.com/s-deojk6ur3d/images/stencil/1280x1280/products/654/3058/gr1wqumwd9lyhkz03i44__39137.1651182564.png?c=1',
        'WARSIA5061'    : 'https://cdn11.bigcommerce.com/s-1z310vvkim/images/stencil/1280x1280/products/231/996/zoaptgufbnqneha4ozv1__89147.1654121181.png?c=1',
        'WARSRB7755'    : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/318/6529/fw91udnci804rjgm0frf__57233.1629813001.png?c=2',
        'WARSRB4041'    : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/152/5999/dn3ploueojp9fpaerpe9__64566.1629295891.png?c=2',
        'WARSRB142'     : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/207/6114/byppnlujuuxjv7tubjqy__84014.1629297293.png?c=2',
        'WARSRB4142'    : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/269/6109/byppnlujuuxjv7tubjqy__64682.1629297257.png?c=2',
        'WARSRB431'     : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/457/5390/abupn1btrsy9jntbarqt__17215.1610052841.png?c=2',
        'WARSRB4310'    : 'https://cdn11.bigcommerce.com/s-azs446wiic/images/stencil/1280x1280/products/457/5390/abupn1btrsy9jntbarqt__17215.1610052841.png?c=2',
        'WARSFS2650'    : 'https://cdn11.bigcommerce.com/s-pqlvcqikb8/images/stencil/1280x1280/products/134/486/wxxkesdo5lnwrrdtwh5n__90608.1642532984.png?c=1',
        'WARSRK751'     : 'https://cdn11.bigcommerce.com/s-deojk6ur3d/images/stencil/1280x1280/products/638/2978/mnacofxulgtnm109o0dm__86427.1651182783.png?c=1',
    }

    image_frame = pd.DataFrame.from_dict(image_dict, orient='index', columns=['ImageName'])

    df_ = pd.merge(
        left = df_,
        right = image_frame,
        left_on = ['icitem'],
        how = 'left',
        right_index = True,
        validate = '1:m',
    )

    return df_ 

In [141]:
def shoe_table(sql_query):
    """Extracts boot information from SQL query. Corresponds with 'Shoe Model' sheet from template

    Args:
        sql_query (DataFrame): DataFrame containined SQL querry results

    Returns:
        DataFrame: DataFrame with boot information
    """
    return (
        sql_query[sql_query['icitem'].str.contains('|'.join(item_list),
                                                   case=False, regex=True)]
        .assign(
            # item number striped of hypen and following characters
            icitem = lambda df_ : df_['icitem'].str.split('-').str[0],
            TempShoeID = '',
            Brand = '',
            SoleImageName = '',
            isANSIApproved = '',
            ItemID = '',
            inUSCollection = '',
            inCanadaCollection = '',
            isASTMapproved = '',
            isDiscontinued = '',
        )
        .pipe(color_select)
        .drop(columns=[
            'icdsc1',
            'icdsc2',
        ])
        .drop_duplicates()
        .pipe(manufacturer_select)
        .pipe(image_merge)
        .rename(columns={
            'icitem'    : 'Style',
            'TempShoeID': 'Id (Temp Shoe ID)'
        })
        .reindex(columns=[
            'Id (Temp Shoe ID)',
            'Style',
            'Manufacturer',
            'Brand',
            'ImageName',
            'SoleImageName',
            'Color',
            'isANSIApproved',
            'ItemID',
            'inUSCollection',
            'inCanadaCollection',
            'isASTMapproved',
            'isDiscontinued',
        ])
        .sort_values(by='Style')
        .reset_index(drop=True)
    )


### Shoe Table Output

In [142]:
shoe_table(sql_query)

,Id (Temp Shoe ID),Style,Manufacturer,Brand,ImageName,SoleImageName,Color,isANSIApproved,ItemID,inUSCollection,inCanadaCollection,isASTMapproved,isDiscontinued
0,,FOOT8125,Avenger,Avenger,https://cdn.shopify.com/s/files/1/1212/1594/pr...,,Brown,,,,,,
1,,FOOT8225,Avenger,Avenger,https://cdn.shopify.com/s/files/1/1212/1594/pr...,,Brown,,,,,,
2,,LPR2002XP,Royer,Royer,https://cdn.shopify.com/s/files/1/0569/8197/93...,,nan,,,,,,
3,,WARSFS2650,Florsheim Work,Florsheim Work,https://cdn11.bigcommerce.com/s-pqlvcqikb8/ima...,,Brown,,,,,,
4,,WARSIA5061,Iron Age,Iron Age,https://cdn11.bigcommerce.com/s-1z310vvkim/ima...,,Brown,,,,,,
5,,WARSRB142,Reebok,Reebok,https://cdn11.bigcommerce.com/s-azs446wiic/ima...,,Black/Grey,,,,,,
6,,WARSRB4041,Reebok,Reebok,https://cdn11.bigcommerce.com/s-azs446wiic/ima...,,Black/Grey,,,,,,
7,,WARSRB4142,Reebok,Reebok,https://cdn11.bigcommerce.com/s-azs446wiic/ima...,,Black/Grey,,,,,,
8,,WARSRB431,Reebok,Reebok,https://cdn11.bigcommerce.com/s-azs446wiic/ima...,,Black/Grey,,,,,,
9,,WARSRB4310,Reebok,Reebok,https://cdn11.bigcommerce.com/s-azs446wiic/ima...,,Black/Grey,,,,,,


## Multi-page Excel report

In [143]:
%%capture --no-display

# Dictionary with sheet names and DataFrames for report
report_data = [
    {
        'sheet_name': 'Shoe Table',
        'report_frame': shoe_table(sql_query)
    },
    {
        'sheet_name': 'Shoe Properties',
        'report_frame': shoe_properties(sql_query)
    },
    {
        'sheet_name': 'Shoe Model',
        'report_frame': shoe_model(sql_query)
    },
]

# Gets the file path to the subdirectory with customer name variable
report_file_path = get_file_path(
    file_name=file_name, file_type="xlsx", customer_name=customer_name)

create_multi_page_report(report_file_path=report_file_path, file_name=file_name,
                         customer_name=customer_name, report_data=report_data)